In [2]:
!pip install python-dotenv
!pip install openai

# Getting data from Mergeflow's API

In [9]:
import requests
import json
import os
from dotenv import load_dotenv # stores API keys for OpenAI and Mergeflow 

load_dotenv()  # Load variables from .env file
mergeflow_api_key = os.getenv('MERGEFLOW_API_KEY') # Assign Mergeflow API

# the API call
query = 'factory automation' # topic is factory automation
dataset = '&q=*%5bfundingcompany%7cfundingproject%7cfundingorganization%5d&sp=2572' # access dataset from mergeflow - in this case venture capital investments
rows = str(10) # first 10 docs only

# in the url below, replace "fw" by your user name. You can see your user name at mergeflow.net
# -> Account -> the string in angle brackets
url = 'https://mergeflow.net/api/v1/ferdinandk/search/getContent?q=' + query + dataset + '&rows=' + rows

# Define the headers to be sent with the request
headers = {
    'MergeflowNet-Api-Auth-Key': mergeflow_api_key
}

# Get and print the response
response = requests.post(url, headers=headers)

mergeflow_docs = []

# Check the response status code and content
if response.status_code == 200:
    mergeflow_response_json = response.json()
    
    # Pretty-print the JSON response -- useful to understand what the JSON looks like
    #pretty_json = json.dumps(mergeflow_response_json, indent=4)
    #print(pretty_json)
    
    # get title, URL, date, and content
    for document in mergeflow_response_json['Documents']:
        current_doc = {
            'title': document['Title'],
            'date': document['Date'],
            'url': document['Url'],
            'content': document['Content']
        }
        
        mergeflow_docs.append(current_doc)
        #print(content)        

    print('Found ' + str(len(mergeflow_docs)) + ' documents.')

else:
    print('Request failed with status code:', response.status_code)
    print(response.text)  # Print the response text


Found 10 documents.


In [17]:
import openai
import os
import requests
import json

openai.api_key = os.getenv('OPENAI_API_KEY')

summary_list = []

system_prompt_summary = f"""
You are a highly qualified academic scholar who has much experience in communicating your results to a broad audience. You are able to extract the most important aspects of texts, compress them and rephrase them in a comprehensible way.
"""

for doc in mergeflow_docs:
    user_prompt_summary = f"""
    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {doc['content']}
    ```
    Please summarize the finding of the text in at most 500 words, and clearly cite the source at the end.
    """
    completion_summary = openai.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[
            {"role": "system", "content": system_prompt_summary},
            {"role": "user", "content": user_prompt_summary}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )

    summary_list.append(completion_summary.choices[0].message.content)
    print(completion_summary.choices[0].message.content)
    print("\n---\n")


print(summary_list)
print("\n----------------------------------------------\n----------------------------------------------\n")


Aniai, a New York City-based robotic company specializing in kitchen solutions, recently secured $12 million in Pre-Series A funding. The funding round was led by InterVest, with contributions from SV Investment, Ignite Innovation from the UK, and existing investor Capstone Partners. The company plans to utilize these funds to establish its first manufacturing facility, named Factory One, in South Korea. This facility will play a crucial role in ensuring stable production and efficient supply chain management for Aniai's flagship product, the Alpha Grill. The move aligns with the company's strategic expansion efforts in North America and globally.

Founded in 2020 and under the leadership of CEO Gunpil Hwang, Aniai focuses on developing robotic solutions for cooking hamburgers. The Alpha Grill, their key product, utilizes robotics to cook hamburgers efficiently. It features a double-sided grill capable of cooking up to eight patties simultaneously or 200 patties per hour. By automating

The Numbers Station, a company based in Menlo Park, California, has recently secured $17.5 million in funding through a series A round and seed funding. This funding was led by Madrona, a prominent venture capital firm. The company specializes in an intelligent data stack automation platform, which aims to streamline data work processes for users. The series A funding specifically amounted to $12.5 million and was significant due to the participation of other key investors such as Norwest Venture Partners, Factory, and notable angel investor Jeff Hammerbacher, who is a co-founder of Cloudera. The total capital raised by Numbers Station now stands at $17.5 million, combining the recent series A round with a prior $5 million seed round. The technology utilized by Numbers Station is based on foundation model technology developed at the Stanford AI lab. This technology allows data workers to more efficiently generate insights by automating repetitive data tasks, initially focusing on data 

The text discusses Software Defined Automation, a company that has recently raised $10 million in seed funding. Insight Partners led the funding round, with participation from Baukunst VC, Fly Ventures, and First Momentum. As a result of the transaction, Jon Rosenbaum from Insight Partners and Axel Bichara from Baukunst will be joining Software Defined Automation's Board of Directors.

Software Defined Automation specializes in innovating factory automation through its Industrial-Control-as-a-Service (ICaaS) offering. This service involves cloud-based management of existing Programmable Logic Controllers (PLCs) known as TechOps, Git-enabled PLC code versioning and collaboration under DevOps, and virtualization of PLCs on edge servers referred to as Virtual PLC. By integrating these technologies, Software Defined Automation aims to dismantle proprietary silos in control technology stacks and facilitate the adoption of API-based modern microservices architecture.

The company is based in

In [12]:
Problem = """Based on the provided information, the problem with the highest overall rating is:
    - Problem: Lack of seamless integration and communication between different factory automation systems and equipment.
    - User Group Impacted: Factory automation engineers and operators.
    - Specific Pain Point: Inefficient workflows, increased downtime, and suboptimal performance due to manual intervention and lack of real-time data synchronization.
    - Summary of Selection Criteria: Large target group impacted, very relevant pain point to factory automation industry and users, specific and critical issue in the industry."""

print(Problem)

Based on the provided information, the problem with the highest overall rating is:
    - Problem: Lack of seamless integration and communication between different factory automation systems and equipment.
    - User Group Impacted: Factory automation engineers and operators.
    - Specific Pain Point: Inefficient workflows, increased downtime, and suboptimal performance due to manual intervention and lack of real-time data synchronization.
    - Summary of Selection Criteria: Large target group impacted, very relevant pain point to factory automation industry and users, specific and critical issue in the industry.


# Answering questions on the results with OpenAI GPT

In [20]:
import openai

openai.api_key = os.getenv('OPENAI_API_KEY') # Assign OpenAI API

#System prompt (for all users)
system_prompt = f"""You are part of a brainstorming session that wants to come up with a solution to a specific problem. You customer handed you the following problem:
       
        {Problem}
        
        Now you try to figure out a solution in a brainstorming session with the Walt Disney Method: 
        It is a tool for brainstorming ideas. It involves using 3 sequential roles, namely the Dreamer, the Realist, and the Critic.
        The Dreamer: you focus on free association and brainstorming of ideas. Anything goes here. It's your chance to let your imagination run wild! Be enthusiastic and approach the task with questions such as “What would be nice?”, “What would be the optimal situation?” or “What would I wish for?” Assume that everything is possible.
        The Realist: Be realistic and decide which of your ideas are actually possible or practical. What would you need to do in the real world in order to make them happen? Take a pragmatic approach and ask which materials, which knowledge, which techniques there are with which the thoughts of the dreamer can be realised.
        The Critic: Examine proposals and its real-world implications from a more critical viewpoint. You want to make sure that your plan is as comprehensive as it can be. Every detail needs to be scrutinized and refined. It is about identifying opportunities and risks, pointing out limits and gaps, and expressing constructive and always positive criticism. Try to identify possible sources of error.
        
        Your work is highly crucial and you are responsible of coming up with the next best innovation.
        """

#Dreamer prompt (idea: come up with crazy ideas)
dreamer_solutions = {}
i = 0

user_prompt = f"""
    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {summary_list}
    ```    
  
    Imagine you are the Dreamer. 
    
    Take a deep breath and work on this problem step by step. 

    
    Please develop 5 really specific ideas/visions on how this problem can be solved. The solutions should consider the following points:
    - You can let your imagination run wild and you do not care about possible limitations. It's more important the idea is new then it can really work.  
    - They should not be too general. 
    - The solution should not already exist on the market and be novel. 
    - The ideas should not be too similar to each other
    
    For the solution, please write only 40 words. It should contain the following contents:
    1) Your solution in max 3 words (title)
    2) Your solution summarized in 1 sentence
    3) Your solution from the customer perspective (customer journey: how does it work for the customer) 
    4) Why the solution is kind of new (not existing on the market)
    5) Please provide some sources of inspiration that you used from the document excerpts.


    
    """
    
completion = openai.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.8,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
dreamer_solutions[i] = completion.choices[0].message
i += 1


#Realist prompt (idea: evaluate if it is possible and what would be needed)
realist_solutions = {}

user_prompt1 = f"""

    Imagine you are the Realist. 
    
    Take a deep breath and work on this problem step by step. 

    
    Consider the following solutions: {dreamer_solutions}.
    
    Please give me the 3 clearly speperated pieces of information for each of the solutions (in total 50 words max):
    1) "Title": Repeat the titel of the idea
    2) "Summary": 3 sentence summary of the solution
    3) "Evaluation criteria 1 detail": What would you need to do in the real world in order to make the solution happen? Answer which knowledge, which techniques there are with which the solutions can be realised. Explain in 5 sentences. 
    4) "Evaluation criteria 1": Is the solution actually possible? Answer can be 1,2 or 3 (1 = not possible, 2 = maybe possible, 3 = possible) (be the most critical person ever. Only give max. 1 of the 3 solution a 3)

    """
    

completion = openai.chat.completions.create(
model="gpt-4-0125-preview",
messages=[
{"role": "system", "content": system_prompt},
{"role": "user", "content": user_prompt1}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

    
print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
realist_solutions[0] = completion.choices[0].message


#Critics prompt (idea: evaluate if there are a lot potential sources of error)
critics_solutions = {}

user_prompt2 = f"""

    Imagine you are the Critic. 
    
    Consider the following solutions: {realist_solutions}.
    
    Take a deep breath and work on this problem step by step. 
    
    Please give me the following 4 clearly seperated pieces of information for each solution (40 words in total):
    1) "Title": Repeat the titel of the idea
    2) "Summary": 3 sentence summary of the solution
    3) "Evaluation criteria 2 details": Identify possible sources of error. Look for risks, pointing out limits and gaps, and expressing constructive criticism. Examine its real-world implications from a more critical viewpoint. You want to make sure that your plan is as comprehensive as it can be. Every detail needs to be scrutinized and refined. (1 sentence only)
    4) "Evaluation criteria 2": Does it have many potential sources of errors? Answer can be 1,2 or 3 (1 = many sources of error, 2 = some sources of error, 3 = few sources of error) (be the most critical person ever. Only give max. 1 of the 3 solution a 3) (1 sentence)
    """
    

completion = openai.chat.completions.create(
model="gpt-4-0125-preview",
messages=[
{"role": "system", "content": system_prompt},
{"role": "user", "content": user_prompt2}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

    
print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
critics_solutions[0] = completion.choices[0].message


#Summary

summary_solutions = {}

user_prompt3 = f"""

    Imagine you should give a summary to a high executive manager about a brainstorming session. 
    
    During the brainstorming session your team came up with the following ideas: {dreamer_solutions}. 
    For each idea you considered how feasible it is {realist_solutions}. 
    Last, you evaluated the amount of potential sources of errors: {critics_solutions} 
    
    Take a deep breath and work on this problem. 
    
    Please summarize in one paragraph for each solution, with the following content:
    
    “Name”: Solution name (5 words max)
    “Solved Problem”: 1 sentence summary of the problem that is being solved 
    “Solution”: Provide a short summary of the solution and how it works (3 sentences max)
    “Impact”: How does the solution solve the specific problem (3 sentences max)
    “Technology”: Please also give a very detailed explanation of which technology is utilized and how it is utilized. Please be very diligent in this task, imagine you are explaining it to the CTO who has a background of engineering and has to implement the solution  (5 sentences max)
    “Sources”: Please provide the specific sources of inspiration that you used from the {summary_list} (1 sentence max)
    
    """
    

completion = openai.chat.completions.create(
model="gpt-4-0125-preview",
messages=[
{"role": "system", "content": system_prompt},
{"role": "user", "content": user_prompt3}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

    
print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
summary_solutions[0] = completion.choices[0].message



1) **Neural Network Mediator**
   - An AI-driven platform facilitating seamless communication between disparate factory systems.
   - Customers integrate their systems once with the platform, enabling automatic real-time data exchange and workflow optimization across diverse equipment.
   - Unique because it uses advanced neural networks to translate and mediate communications between incompatible systems, learning and adapting over time.
   - Inspired by Aniai's use of cloud-based AI for enhancing kitchen productivity and Cellares' end-to-end automation in cell therapy manufacturing.

2) **Quantum Integration Protocol**
   - Quantum computing-based protocol ensuring instant, secure data integration across factory automation systems.
   - Factories implement this protocol to achieve unprecedented data processing speeds and security, making integration effortless and instantaneous.
   - Novel due to its application of quantum computing principles to solve integration challenges, a leap 

**Neural Network Mediator**

- **Solved Problem**: Enhances communication between incompatible factory systems.
- **Solution**: This AI-driven platform utilizes neural networks to facilitate real-time data exchange and workflow optimization, enabling systems to 'speak' a common language. It translates and mediates communications between disparate systems, learning and adapting to optimize integration over time. The goal is to minimize manual intervention and improve overall factory efficiency.
- **Impact**: By ensuring seamless integration, the platform reduces downtime and inefficiencies, leading to optimized operations and productivity. It adapts to changes and scales with the factory's evolving needs, promoting long-term sustainability and competitiveness. Moreover, it streamlines workflows, significantly reducing the potential for human error and system miscommunication.
- **Technology**: The platform employs advanced neural networks that analyze, learn, and adapt to the languages 

In [ ]:
# Initial rating of the solutions

initial_rating = {}

system_prompt_2 = f"""You are a highly qualified critical rater of business opportunities. You are now tasked to evaluate a set of opportunities with the following problem in mind: {Problem} """


user_prompt4 = f""""

Before you give me any output, please rate the following solutions: {summary_solutions}. Rate according to the following criteria. Please be very critical in your ratings and only give a 3 out of 3 rating when it is very good:   

    - Functionality and Performance: Assess whether the solution addresses and solves the problem it’s supposed to solve. This includes estimating if the solution can solve the problem quickly and effectively. (1 = low performance, 2 = medium performance, 3 = high performance)
    - Scalability: Evaluate whether the solution can adapt to future growth (or shrinking) of demand and accommodate changes in requirements: How flexible is the solution? (1 = low scalability, 2 = medium scalability, 3 = high scalability)
    - Cost-effectiveness: Assess the costs associated with the solution. How large are upfront costs for implementation such as training, facilities, machines, etc.? How large are operating costs such as personnel, energy, maintenance, etc.? The solution should have a relatively high return on investment. (1 = low cost-effectiveness, 2 = medium cost-effectiveness 3 = high cost-effectiveness)
    - Innovativeness: How innovative is the solution? Do similar solutions for the problem exist already? (1 = low innovativeness, 2 = medium innovativeness 3 = high innovativeness)
    - Uniqueness: The solutions contained in the list should differ substantially and not be similar. (1 = low uniqueness, 2 = medium uniqueness 3 = high uniqueness)

Please repeat the name of the solution and then output the ratings and a one-sentence justification (max 20 words). Add up the 5 different ratings you gave to an overall score out of 15 for each solution.

"""


completion = openai.chat.completions.create(
model="gpt-4-0125-preview",
messages=[
{"role": "system", "content": system_prompt_2},
{"role": "user", "content": user_prompt4}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
initial_rating[0] = completion.choices[0].message



In [ ]:
# Improvement loop

improved_ideas= {}

system_prompt_3 = """You are a CTO with extensive experience in improving innovative ideas."""

user_prompt5 = f""""
        You have the following solutions: {summary_solutions}.
        The corresponding ratings are in: {initial_rating}.
        
        Please iterate each idea in order to strongly improve it's: 
            - Functionality and Performance: Assess whether the solution addresses and solves the problem it’s supposed to solve. This includes estimating if the solution can solve the problem quickly and effectively. 
            - Scalability: Evaluate whether the solution can adapt to future growth (or shrinking) of demand and accommodate changes in requirements: How flexible is the solution?
            - Cost-effectiveness: Assess the costs associated with the solution. How large are upfront costs for implementation such as training, facilities, machines, etc.? How large are operating costs such as personnel, energy, maintenance, etc.? The solution should have a relatively high return on investment.
            - Innovativeness: How innovative is the solution? Do similar solutions for the problem exist already?
            - Uniqueness: The solutions contained in the list should differ substantially and not be similar.
            
       For each of the solutions summarize the solution with the following content (keep the same order as in the input):
    
        “Name”: Solution name (5 words max)
        “Solved Problem”: 1 sentence summary of the problem that is being solved 
        “Solution”: Provide a short summary of the solution and how it works (3 sentences max)
        “Impact”: How does the solution solve the specific problem (3 sentences max)
        “Technology”: Please also give a very detailed explanation of which technology is utilized and how it is utilized. Please be very diligent in this task, imagine you are explaining it to the CTO who has a background of engineering and has to implement the solution  (5 sentences max)
        “Sources”: Please provide the specific sources of inspiration that you used from the {summary_list} (1 sentence max)
    
        """

completion = openai.chat.completions.create(
model="gpt-4-0125-preview",
messages=[
{"role": "system", "content": system_prompt_3},
{"role": "user", "content": user_prompt5}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
improved_ideas[0] = completion.choices[0].message


In [44]:
# Final rating of the solutions


system_prompt_3 = """You are a CTO with extensive experience in improving innovative ideas."""

user_prompt_6 = f""""

Before you give me any output, please rate the following solutions: {improved_ideas}. Rate according to the following criteria. Please be very critical in your ratings and only give a 3 out of 3 rating when it is very good:   

    - Functionality and Performance: Assess whether the solution addresses and solves the problem it’s supposed to solve. This includes estimating if the solution can solve the problem quickly and effectively. (1 = low performance, 2 = medium performance, 3 = high performance)
    - Scalability: Evaluate whether the solution can adapt to future growth (or shrinking) of demand and accommodate changes in requirements: How flexible is the solution? (1 = low scalability, 2 = medium scalability, 3 = high scalability)
    - Cost-effectiveness: Assess the costs associated with the solution. How large are upfront costs for implementation such as training, facilities, machines, etc.? How large are operating costs such as personnel, energy, maintenance, etc.? The solution should have a relatively high return on investment. (1 = low cost-effectiveness, 2 = medium cost-effectiveness 3 = high cost-effectiveness)
    - Innovativeness: How innovative is the solution? Do similar solutions for the problem exist already? (1 = low innovativeness, 2 = medium innovativeness 3 = high innovativeness)
    - Uniqueness: The solutions contained in the list should differ substantially and not be similar. (1 = low uniqueness, 2 = medium uniqueness 3 = high uniqueness)

Please repeat the name of the solution and then output the ratings and a one-sentence justification (max 20 words). Add up the 5 different ratings you gave to an overall score out of 15 for each solution.

"""


completion = openai.chat.completions.create(
model="gpt-4-0125-preview",
messages=[
{"role": "system", "content": system_prompt_3},
{"role": "user", "content": user_prompt_6}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
initial_rating[0] = completion.choices[0].message




### Enhanced Quantum Integration Protocol

- **Functionality and Performance**: 3 - Significantly enhances data integration speed and security using quantum computing.
- **Scalability**: 3 - Quantum computing offers unparalleled scalability for future demands.
- **Cost-effectiveness**: 2 - High initial costs but offers substantial long-term savings and security benefits.
- **Innovativeness**: 3 - Leverages cutting-edge quantum technology for a novel approach to data integration.
- **Uniqueness**: 3 - Sets a new benchmark in integration protocols with quantum computing.

**Overall Score**: 14/15

### Cost-Effective Blockchain Sync Network

- **Functionality and Performance**: 3 - Offers real-time, secure data synchronization across systems.
- **Scalability**: 3 - Blockchain technology inherently supports scalability through its distributed ledger system.
- **Cost-effectiveness**: 2 - While setup costs can be high, operational costs and ROI are favorable.
- **Innovativeness**: 2 - Applie